## NOTEBOOK IMPORTS

In [1]:
import numpy as np
import pandas as pd

## CONSTANSTS

In [2]:
TRAINING_DATA_FILE = './SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = './SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = './SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = './SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = './SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = './SpamData/03_Testing/test-features.txt'
TEST_TARGET = './SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

## READ AND LOAD FEATURES FROM .txt FILE INTO NUMPY ARRAY

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [5]:
sparse_train_data[-5:]

array([[5795, 2036,    0,    1],
       [5795, 2040,    0,    2],
       [5795, 2272,    0,    1],
       [5795, 2343,    0,    1],
       [5795, 2368,    0,    1]])

In [6]:
print('Number of rows in Training data:', sparse_train_data.shape[0])
print('Number of rows in Test data:', sparse_test_data.shape[0])

Number of rows in Training data: 258361
Number of rows in Test data: 117781


In [7]:
print('Number of emails in Training data:', np.unique(sparse_train_data[:, 0]).size)

Number of emails in Training data: 4013


In [8]:
print('Number of emails in Test data:', np.unique(sparse_test_data[:, 0]).size)

Number of emails in Test data: 1724


In [9]:
np.unique(sparse_train_data[:, 0]).shape

(4013,)

## HOW TO CREATE AN EMPTY DATAFRAME

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))

In [11]:
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [12]:
index_names = np.unique(sparse_train_data[:,0])

In [13]:
full_train_matrix = pd.DataFrame(columns=column_names, index=index_names)
full_train_matrix.fillna(value=0, inplace=True)

In [14]:
full_train_matrix.tail()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## CREATE A FULL MATRIX FROM SPARSE MATRIX

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    
    '''
        Form a full matrix from a sparse matrix. Return a pandas dataframe.
        
        keyword arguments:
        sparse_matrix -- numpy array
        nr_words -- size of vocabulary . Total number of tokens.
        doc_idx -- position of document id in the sparse matrix. Default: 1st column
        word_idx -- position of the word id in the sparse matrix. default: 2nd column
        cat_idx -- position of the label (spam is 1, non-spam is 0). Default: 3rd column
        freq_word -- position of occurance of word in sparse matrix. Default: 4th column
        
    '''
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    
    full_matrix = pd.DataFrame(columns=column_names, index=doc_id_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurance = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurance
        
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix

In [16]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 22.5 s


In [17]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
sparse_train_data.shape

(258361, 4)

## PROBABILITY OF SPAM AND NON SPAM

In [19]:
# SPAM EMAILS
prob_spam = full_train_data.CATEGORY[full_train_data.CATEGORY == 1].value_counts() / full_train_data.CATEGORY.shape[0]
prob_spam

1    0.310989
Name: CATEGORY, dtype: float64

In [20]:
# NON-SPAM EMAILS
prob_ham = full_train_data.CATEGORY[full_train_data.CATEGORY == 0].value_counts() / full_train_data.CATEGORY.shape[0]
prob_ham

0    0.689011
Name: CATEGORY, dtype: float64

## TOTAL NUMBER OF WORDS / TOKENS

In [21]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']

In [22]:
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
email_lenghts = full_train_features.sum(axis=1)
email_lenghts.shape

(4013,)

In [24]:
total_wc = email_lenghts.sum()
total_wc

429221

## NUMBER OF TOKENS IN SPAM AND HAM EMAILS

In [25]:
spam_lenghts = email_lenghts[full_train_data.CATEGORY == 1]

In [26]:
spam_lenghts.shape

(1248,)

In [27]:
# SPAM WORD COUNT
spam_wc = spam_lenghts.sum()
spam_wc

176346

In [28]:
ham_lenghts = email_lenghts[full_train_data.CATEGORY == 0]

In [29]:
ham_lenghts.shape

(2765,)

In [30]:
# HAM WORD COUNT
ham_wc = ham_lenghts.sum()
ham_wc

252875

In [31]:
email_lenghts.shape[0] - spam_lenghts.shape[0] - ham_lenghts.shape[0]

0

In [32]:
ham_lenghts.max()

6869

In [33]:
spam_lenghts.max()

4882

In [34]:
# which spam or Non-spam emails tend to be longer ?
# Which category contains more tokens ?

print('Average Number of words in Spam emails:', round(spam_lenghts.sum()/spam_lenghts.shape[0]))
print('Average Number of words in Ham Emails:', round(ham_lenghts.sum()/ham_lenghts.shape[0]))

Average Number of words in Spam emails: 141
Average Number of words in Ham Emails: 91


## SUM OF THE TOKENS OCCURING IN SPAM

In [35]:
full_train_features.shape

(4013, 2500)

In [36]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
train_spam_tokens.shape

(1248, 2500)

In [38]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1

In [39]:
summed_spam_tokens.shape

(2500,)

In [40]:
summed_spam_tokens.tail()

2495     2
2496    10
2497    13
2498    20
2499     6
dtype: int64

## SUM OF TOKENS OCCURING IN HAM EMALS

In [41]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
train_ham_tokens.shape

(2765, 2500)

In [43]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [44]:
summed_ham_tokens.shape

(2500,)

In [45]:
summed_ham_tokens.tail()

2495    22
2496    22
2497    12
2498     5
2499    25
dtype: int64

## P(Token | Spam) - Probability that a token Occurs given the email is spam

In [46]:
# PROBALITY OCCURANCE OF TOKENS IN SPAM DATA
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)

In [47]:
prob_tokens_spam[:5]

0    0.012184
1    0.005228
2    0.006805
3    0.011306
4    0.006816
dtype: float64

In [48]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a token Occurs given the email is Non-spam

In [49]:
prob_tokens_nonspam = summed_ham_tokens / (ham_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

0.9999999999999999

## P(Token) - Probability that Token Occurs

In [50]:
# PROBABILITY OF ALL OCCURANCE OF THE TOKENS
prob_token_all = full_train_features.sum(axis=0) / total_wc

In [51]:
prob_token_all.sum()

1.0

## SAVE THE TRAINED MODEL

In [52]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam, delimiter=' ')
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam, delimiter=' ')
np.savetxt(TOKEN_ALL_PROB_FILE, prob_token_all, delimiter=' ')

## PREPARE TEST DATA
Create a Full Matrix from the sparse_test_data. Time the function  call, How long it takes. Separate the features and the target values. Save the as separate .txt files: a TEST_TARGET and a TEST_FEATURE_MATRIX file.

In [53]:
sparse_test_data.shape

(117781, 4)

In [54]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 10.4 s


In [55]:
full_test_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [57]:
np.savetxt(TEST_FEATURE_MATRIX, X_test)
np.savetxt(TEST_TARGET, y_test)